# Examen ETL: SPARK 02/02

Se podrá utilizar toda la información que se encuentra en el campus. 

El fichero de datos sobre el que se trabajará es el de partidosLigaNBA.txt.

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

### Mª del Val Huerta Pintado

## Primera pregunta: Describe brevemente que diferencia el persists, cache y collect en spark. Explica brevemente casos en los que es interesante su aplicación

__Persists y Caché:__ Son técnicas de optimización para cálculos (iterativos e interactivos) de Spark. Permiten guardar resultados para ser reutilizados en etapas posteriores. Son muy útiles  si se quiere __acceder repetidamente a los datos o si se quiere consultar un pequeño conunto de datos o cuando se ejecuta un algoritmo iterativo__. La diferencia entre cache() y persist() es que usando cache() el nivel de almacenamiento por defecto es MEMORY_ONLY mientras que usando persist() podemos usar varios niveles de almacenamiento.


__Collect__: Devuelve todos los elementos del conjunto de datos como una matriz en el programa del controlador. Esto suele ser útil después de un filtro u otra operación que devuelve un subconjunto suficientemente pequeño de los datos ya que si no aplicasemos antes el filtro por ejemplo, nos devolverá todo el conjunto de datos al controlador y eso puede conllevar la pérdida de memoria. 

## Segunda pregunta: Explica brevemente los pasos a seguir para realizar la carga de un conjunto de datos (pasos que se siguieron en la práctica con datos de logs)

Para comenzar hay que cargar las librerías, después hay que cargar los datos en un contexto Spark, este es un paso muy importante. Entonces leemos nuestros datos mediante la ruta pertinente y los introducimos en nuestro contexto con SparkContext().

## Tercera Pregunta: Índica un tipo de problema que puede empeorar los datos. (pe. Que no exista un representante del CDO en todas las áreas de negocio), pon algún ejemplo específico (pe. Datos duplicados) y cómo lo tratarías con técnicas de data cleaning.

Uno de los ejemplos que puede suponer un problema a la hora de analizar los datos puede ser:
* Que el texto haya que reformatearlo o reemplazarlo
* Los cálculos estén mal realizados 
* Los datos estén incompletos  
* El tipo de datos no esté en el formato correcto : Por ejemplo las fechas. 
* Filas duplicadas
* Outliers
* Datos desordenados

## Cuarta tarea: Inicializar spark context y cargar los datos desde el fichero.

Primero importamos los datos en SparkContext.

In [1]:
from pyspark import SparkContext
sc = SparkContext("local", "First App")

In [2]:
data_file = "./partidosLigaNBA.csv"
datos = sc.textFile(data_file)

In [3]:
datos.count()

12908

Una vez descargado el fichero podemos generar un RDD que lo hemos llamado "datos"

In [4]:
datos.take(5)

['Date:Start..ET.:Visitor.Neutral:PTS:Home.Neutral:PTS.1',
 'Tue, Oct 30, 2007:"7:30 pm":Utah Jazz:117:Golden State Warriors:96',
 'Tue, Oct 30, 2007:"7:30 pm":Houston Rockets:95:Los Angeles Lakers:93',
 'Tue, Oct 30, 2007:"7:00 pm":Portland Trail Blazers:97:San Antonio Spurs:106',
 'Wed, Oct 31, 2007:"8:00 pm":Dallas Mavericks:92:Cleveland Cavaliers:74']

##### Limpieza de datos

Tenemos que realizar un split porque vienen los datos unidos y eliminar el header, esta segunda es porque voy a trabajar sobre las columnas por lo que como el header son letras me dificultaria el trabajo. 

In [5]:
#Eliminamos el header
header = datos.filter(lambda x: 'Visitor.Neutral' in x)

In [6]:
datos_sin_header = datos.subtract(header) #de mis datos sustraeme el header que es la primera linea 

In [7]:
datos_sin_header.take(5)

['Tue, Jan 12, 2016:"8:00 pm":Oklahoma City Thunder:101:Minnesota Timberwolves:96',
 'Sat, Jan 12, 2008:"7:30 pm":Boston Celtics:78:Washington Wizards:85',
 'Wed, Apr 16, 2008:"7:00 pm":New York Knicks:123:Indiana Pacers:132',
 'Fri, Dec 26, 2008:"7:30 pm":Minnesota Timberwolves:120:New York Knicks:107',
 'Wed, Nov 6, 2013:"9:30 pm":Dallas Mavericks:93:Oklahoma City Thunder:107']

In [8]:
datos_limpios = datos_sin_header.map(lambda x: x.split(":"))#Separar datos
datos_limpios.take(5)

[['Tue, Jan 12, 2016',
  '"8',
  '00 pm"',
  'Oklahoma City Thunder',
  '101',
  'Minnesota Timberwolves',
  '96'],
 ['Sat, Jan 12, 2008',
  '"7',
  '30 pm"',
  'Boston Celtics',
  '78',
  'Washington Wizards',
  '85'],
 ['Wed, Apr 16, 2008',
  '"7',
  '00 pm"',
  'New York Knicks',
  '123',
  'Indiana Pacers',
  '132'],
 ['Fri, Dec 26, 2008',
  '"7',
  '30 pm"',
  'Minnesota Timberwolves',
  '120',
  'New York Knicks',
  '107'],
 ['Wed, Nov 6, 2013',
  '"9',
  '30 pm"',
  'Dallas Mavericks',
  '93',
  'Oklahoma City Thunder',
  '107']]

## Quinta tarea: Media de la diferencia de puntos por año

In [9]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [10]:
from pyspark.sql import Row
import datetime as dt

datos_limpios = datos_sin_header.map(lambda x: x.split(":")) #SEPARAMOS LOS DATOS
dataframe = datos_limpios.map(lambda p: Row(
    fecha = dt.datetime.strptime((p[0]),'%a, %b %d, %Y').strftime('%m/%d/%Y'), 
    hora = p[1] + ':' + p[2],
    equipo_local = p[3],
    equipo_visitante = p[4],
    puntos_local = int(p[5]),
    puntos_visitante = int(p[6])
    )
)

In [11]:
#PREGUNTAR
interactions_df = sqlContext.createDataFrame(dataframe)
interactions_df.registerTempTable("interactions")

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 6, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/pyspark/rdd.py", line 1306, in takeUpToNumLeft
    yield next(iterator)
  File "<ipython-input-10-eebfea6dd1af>", line 10, in <lambda>
ValueError: invalid literal for int() with base 10: 'Minnesota Timberwolves'

	at or

## Sexta tarea: ¿Han judado todos los equipos el mismo número de partidos? ¿ Si es qué no a que puede deberse?

In [12]:
from operator import add 

Extraigo el número de partidos que han jugado los equipos separándolos en local y visitiante y así obtengo el número de partidos que ha jugado un mismo equipo como local o como visitante 

In [16]:
visiantes = datos_limpios.map(lambda a: (a[5], 1))
locales = datos_limpios.map(lambda a: (a[3], 1))



visitantes_agrupados = visiantes.reduceByKey(add).sortBy(lambda b: b[1], False).collect() #Reduceme con esta clave y ordenamelo
locales_agrupados = locales.reduceByKey(add).sortBy(lambda y: y[1], False).collect()


In [ ]:
#PREGUNTAR por qué esto no es posible (cambiar el add por lambda a,b)
visitantes_agrupados = visiantes.reduceByKey(lambda a,b = a+b).collect().sortBy(lambda b: b[1], False).collect()

In [17]:
visitantes_agrupados

[('San Antonio Spurs', 467),
 ('Boston Celtics', 467),
 ('Miami Heat', 461),
 ('Los Angeles Lakers', 450),
 ('Cleveland Cavaliers', 449),
 ('Atlanta Hawks', 448),
 ('Golden State Warriors', 445),
 ('Chicago Bulls', 436),
 ('Houston Rockets', 434),
 ('Indiana Pacers', 434),
 ('Memphis Grizzlies', 433),
 ('Orlando Magic', 431),
 ('Los Angeles Clippers', 431),
 ('Dallas Mavericks', 431),
 ('Toronto Raptors', 426),
 ('Portland Trail Blazers', 425),
 ('Denver Nuggets', 424),
 ('Utah Jazz', 422),
 ('Washington Wizards', 421),
 ('Philadelphia 76ers', 416),
 ('Detroit Pistons', 415),
 ('Milwaukee Bucks', 413),
 ('Phoenix Suns', 412),
 ('New York Knicks', 412),
 ('Oklahoma City Thunder', 410),
 ('Minnesota Timberwolves', 402),
 ('Sacramento Kings', 402),
 ('Charlotte Bobcats', 283),
 ('New Orleans Hornets', 250),
 ('Brooklyn Nets', 217),
 ('New Jersey Nets', 197),
 ('New Orleans Pelicans', 166),
 ('Charlotte Hornets', 126),
 ('Seattle SuperSonics', 41),
 ('Playoffs', 10)]

In [18]:
locales_agrupados

[('San Antonio Spurs', 466),
 ('Boston Celtics', 463),
 ('Miami Heat', 456),
 ('Cleveland Cavaliers', 452),
 ('Atlanta Hawks', 448),
 ('Los Angeles Lakers', 447),
 ('Golden State Warriors', 440),
 ('Chicago Bulls', 437),
 ('Dallas Mavericks', 436),
 ('Memphis Grizzlies', 434),
 ('Indiana Pacers', 434),
 ('Houston Rockets', 434),
 ('Orlando Magic', 432),
 ('Los Angeles Clippers', 430),
 ('Portland Trail Blazers', 428),
 ('Utah Jazz', 424),
 ('Toronto Raptors', 424),
 ('Denver Nuggets', 424),
 ('Washington Wizards', 423),
 ('Philadelphia 76ers', 418),
 ('Milwaukee Bucks', 414),
 ('Detroit Pistons', 414),
 ('Phoenix Suns', 413),
 ('New York Knicks', 413),
 ('Oklahoma City Thunder', 408),
 ('Minnesota Timberwolves', 402),
 ('Sacramento Kings', 402),
 ('Charlotte Bobcats', 283),
 ('New Orleans Hornets', 249),
 ('Brooklyn Nets', 218),
 ('New Jersey Nets', 197),
 ('New Orleans Pelicans', 166),
 ('Charlotte Hornets', 127),
 ('Seattle SuperSonics', 41),
 ('Playoffs', 10)]

## Séptima pregunta: ¿Cuantos partidos ha ganado en Enero Clevelant?

In [19]:
#Jugando como local
clevelant_local = datos_limpios.filter(lambda x:x[3] == 'Cleveland Cavaliers').filter(lambda x:'Jan' in x[0]).filter(lambda x: int(x[4])<int(x[6]))
clevelant_local.count()

44

In [20]:
#Jugando como visitante
clevelant_visitante = datos_limpios.filter(lambda x:x[5] == 'Cleveland Cavaliers').filter(lambda x:'Jan' in x[0]).filter(lambda x: int(x[4])<int(x[6]))
clevelant_visitante.count()

42

In [21]:
#Total de partidos ganados
clevelant_local.count() + clevelant_visitante.count()

86

## Octava pregunta: ¿Los Warrios son mejores fuera de casa o en casa?

In [65]:
#Ganado como local
warrior_local = datos_limpios.filter(lambda x: x[3] == 'Golden State Warriors').filter(lambda x: int(x[4])>int(x[6])).map(lambda x: (x[3],1)).reduceByKey(add).sortByKey().take(1)

In [67]:
warrior_visitante = datos_limpios.filter(lambda x:x[5] == 'Golden State Warriors').filter(lambda x:int(x[6])>int(x[4])).map(lambda x : (x[5],1)).reduceByKey(add).sortByKey().take(1)

In [69]:
warrior_local + warrior_visitante

[('Golden State Warriors', 215), ('Golden State Warriors', 308)]

## Novena pregunta: Equipo que ha quedado primerio en victorias más temporadas. (si es que hay alguno que más)

##  Décima pregunta: Escribe la expresión regular correcta que sólo macheen los teléfonos y el correo del siguiente texto.

Si eres cliente y necesitas información sobre tus posiciones, productos o realizar operaciones: Desde España. Desde el extranjero. Banca telefónica en castellano. Bandera castellano. 902 13 23 13. Banca telefónica en catalán. Bandera catalana. 902 88 30 08. Banca telefónica en inglés. Bandera inglesa. 902 88 88 35. O por correo electrónico a atencioncliente@bankinter.com